# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from pathlib import Path
import holoviews as hv

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
csv_file = Path.cwd().parent / "WeatherPy/output_data/cities.csv"
city_data_df = pd.read_csv(csv_file)

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,kruisfontein,-34.0033,24.7314,26.05,56,44,5.60,ZA,1705144880
1,1,prince rupert,54.3161,-130.3201,-9.95,73,40,2.57,CA,1705144881
2,2,invercargill,-46.4000,168.3500,14.45,94,99,4.92,NZ,1705144883
3,3,blackmans bay,-43.0167,147.3167,13.74,56,1,1.34,AU,1705144885
4,4,puerto ayora,-0.7393,-90.3518,24.75,95,3,2.68,EC,1705144886


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
city_map = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    global_extent=True,
    frame_height = 700,
    size = "Humidity",
    scale = 0.7,
    color = "City",
    alpha = 0.5
)
# Display the map

city_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
max_temp = 28
min_temp = 22
max_wind_speed = 6
max_clouds = 25
max_humidity = 80

nice_city_df = city_data_df.loc[
    (city_data_df["Humidity"] <= max_humidity) &
    (city_data_df["Max Temp"] <= max_temp) &
    (city_data_df["Max Temp"] >= min_temp) &
    (city_data_df["Wind Speed"] < max_wind_speed) &
    (city_data_df["Cloudiness"] < max_clouds), :
]

# Drop any rows with null values
nice_city_df.dropna(how = 'any')

# Display sample data
nice_city_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
18,18,salgueiro,-8.0742,-39.1192,26.71,66,13,4.98,BR,1705144898
39,39,pitimbu,-7.4706,-34.8086,27.33,72,2,3.94,BR,1705144911
47,47,montes claros,-16.7350,-43.8617,25.88,57,0,2.57,BR,1705144917
57,57,mangrol,21.1167,70.1167,27.51,41,0,5.91,IN,1705144923
93,93,port elizabeth,-33.9180,25.5701,26.22,73,20,5.66,ZA,1705144947
161,161,hermanus,-34.4187,19.2345,22.74,73,0,4.52,ZA,1705144994
194,194,belmonte,-15.8631,-38.8828,27.99,67,1,3.14,BR,1705145018
198,198,san antonio de pale,-1.4014,5.6325,26.92,76,18,4.32,GQ,1705145021
220,220,paka,4.6374,103.4368,27.96,74,20,3.09,MY,1705145036
222,222,cabedelo,-6.9811,-34.8339,27.37,73,20,4.41,BR,1705144853


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = nice_city_df.loc[:, ["City", "Country", "Lat", "Lng", "Humidity"]]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
18,salgueiro,BR,-8.0742,-39.1192,66,
39,pitimbu,BR,-7.4706,-34.8086,72,
47,montes claros,BR,-16.7350,-43.8617,57,
57,mangrol,IN,21.1167,70.1167,41,
93,port elizabeth,ZA,-33.9180,25.5701,73,
161,hermanus,ZA,-34.4187,19.2345,73,
194,belmonte,BR,-15.8631,-38.8828,67,
198,san antonio de pale,GQ,-1.4014,5.6325,76,
220,paka,MY,4.6374,103.4368,74,
222,cabedelo,BR,-6.9811,-34.8339,73,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000

params = {
    "filter":"",
    "bias":"",
    "apiKey":geoapify_key,
    "categories":"accommodation.hotel"  
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    longitude = hotel_df.loc[index, "Lng"]
    latitude = hotel_df.loc[index, "Lat"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
salgueiro - nearest hotel: Mustang Hotel
pitimbu - nearest hotel: Pousada Brisa Mar
montes claros - nearest hotel: Hotel Alvorada
mangrol - nearest hotel: No hotel found
port elizabeth - nearest hotel: Waterford Hotel
hermanus - nearest hotel: Aloe guest house
belmonte - nearest hotel: No hotel found
san antonio de pale - nearest hotel: No hotel found
paka - nearest hotel: Paka Inn
cabedelo - nearest hotel: No hotel found
cidreira - nearest hotel: Hotel Castelo
itamaraca - nearest hotel: No hotel found
santo amaro - nearest hotel: Hotel da Purificação
taoudenni - nearest hotel: No hotel found
mokhotlong - nearest hotel: Mokhotlong Hotel
mana - nearest hotel: Le Samana
pathein - nearest hotel: Taan Taan Ta
chiang rai - nearest hotel: Le Méridien Chiang Rai Resort
kutum - nearest hotel: No hotel found
minbu - nearest hotel: Rolex Guesthouse
mermoz boabab - nearest hotel: La maison blanche


,City,Country,Lat,Lng,Humidity,Hotel Name
18,salgueiro,BR,-8.0742,-39.1192,66,Mustang Hotel
39,pitimbu,BR,-7.4706,-34.8086,72,Pousada Brisa Mar
47,montes claros,BR,-16.7350,-43.8617,57,Hotel Alvorada
57,mangrol,IN,21.1167,70.1167,41,No hotel found
93,port elizabeth,ZA,-33.9180,25.5701,73,Waterford Hotel
161,hermanus,ZA,-34.4187,19.2345,73,Aloe guest house
194,belmonte,BR,-15.8631,-38.8828,67,No hotel found
198,san antonio de pale,GQ,-1.4014,5.6325,76,No hotel found
220,paka,MY,4.6374,103.4368,74,Paka Inn
222,cabedelo,BR,-6.9811,-34.8339,73,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    global_extent=True,
    frame_height = 800,
    size = "Humidity",
    hover_cols = ["Hotel Name", "Country"],
    scale = 0.7,
    color = "City",
    alpha = 0.5
)

# Display the map
hotel_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)